In [4]:
from datasets import load_dataset
import os

In [7]:
data_path = "/data/VLMGroundingProject/Datasets/InstructPart/train1800/instructpart_train_dataset_for_segzero_partfield_840"
data_split = "train"    

In [ ]:
if os.path.isdir(data_path):
    # when we use dataset builder, we should always refer to the train split
    file_type = os.path.splitext(os.listdir(data_path)[0])[-1][1:].replace("jsonl", "json")
    print(file_type)
    dataset = load_dataset(file_type, data_dir=data_path, split=data_split)
elif os.path.isfile(data_path):
    file_type = os.path.splitext(data_path)[-1][1:].replace("jsonl", "json")
    dataset = load_dataset(file_type, data_files=data_path, split=data_split)
else:
    # load remote dataset from huggingface hub
    dataset = load_dataset(data_path, split=data_split)

json


Generating train split: 0 examples [00:00, ? examples/s]Failed to load JSON from file '/data/VLMGroundingProject/Datasets/InstructPart/train1800/instructpart_train_dataset_for_segzero_partfield_840/train/data-00000-of-00003.arrow' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Invalid value. in row 0
Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [10]:
file_type = os.path.splitext(os.listdir(data_path)[0])[-1][1:].replace("jsonl", "json")

In [11]:
file_type

'json'

In [12]:
os.listdir(data_path)[0]

'dataset_dict.json'

In [13]:
dataset = load_dataset(data_path, split=data_split)

Generating train split: 1800 examples [00:02, 710.04 examples/s]


In [15]:
type(dataset[0])

dict

In [17]:
dataset[69]

{'id': '5495032190_9998ec0e96_o',
 'problem': "scissors' handle",
 'solution': '[{"bbox_2d": [181, 119, 360, 249], "point_2d": [271, 184]}, {"bbox_2d": [49, 329, 362, 624], "point_2d": [206, 477]}]',
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=840x840>,
 'img_height': 1500,
 'img_width': 2000,
 'object_part': True}

In [24]:
format_prompt = "/home/ksmehrab/AttentionGrounding/ModelPlaygrounds/SegZero/EvaluationScripts/Prompts/vrpart2_prompt.txt"

s_format_prompt = None
s_format_in_jinja = False
if format_prompt:
    if format_prompt.endswith(".jinja"):
        s_format_in_jinja = True
    with open(format_prompt, encoding="utf-8") as f:
        s_format_prompt = f.read()

s_prompt_key = "problem"
s_image_key = "image"
example = dataset[0]
prompt_str: str = example[s_prompt_key]
if s_format_prompt:
    if s_format_in_jinja:
        format_prompt = Template(s_format_prompt.strip())
        prompt_str = format_prompt.render(content=prompt_str)
    else:
        # expects a format string with "Question" variable
        prompt_str = s_format_prompt.format(Question=prompt_str.lower().strip(".")).strip()

if (s_image_key in example) and (not prompt_str.startswith("<image>")):
    prompt_str = "<image>" + prompt_str

In [26]:
if s_image_key in example:
    # https://huggingface.co/docs/transformers/en/tasks/image_text_to_text
    content_list = []
    for i, content in enumerate(prompt_str.split("<image>")):
        if i != 0:
            content_list.append({"type": "image"})

        if content:
            content_list.append({"type": "text", "text": content})

    res= [{"role": "user", "content": content_list}]